In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [0]:
%matplotlib inline
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

os.chdir('/content/drive/My Drive/GitHub Repositories/Tuberculosis/Tuberculosis Classification')

trainDir = './data/Train'
validDir = './data/Test'

IMG_SIZE = 512 # slightly smaller than vgg16 normally expects
datagen = ImageDataGenerator(rescale=1./255, 
                             samplewise_center=False, 
                             samplewise_std_normalization=False, 
                             horizontal_flip = True, 
                             vertical_flip = False, 
                             height_shift_range = 0.15, 
                             width_shift_range = 0.15, 
                             rotation_range = 5, 
                             shear_range = 0.01,
                             fill_mode = 'nearest',
                             zoom_range=0.1,  
                             )

train_gen = datagen.flow_from_directory(
    directory=trainDir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=2019)

valid_gen = datagen.flow_from_directory(
    directory=validDir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    seed=2019)




In [0]:
import matplotlib.pyplot as plt

t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(2, 4, figsize = (16, 8))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = 0, vmax = 1)
    c_ax.set_title('%s' % ('Pulmonary Abnormality' if np.argmax(c_y)>0.5 else 'Healthy'))
    c_ax.axis('off')


In [0]:
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.models import *

IMG_SIZE = 512
img_shape = (IMG_SIZE, IMG_SIZE, 3)
inputs = Input(img_shape)
num_classes = 2

vgg16_model = VGG16(input_shape =  img_shape, include_top = False, weights = 'imagenet')
vgg16_model.trainable = False
outputs = vgg16_model(inputs)
outputs = BatchNormalization(name = 'BatchNormalization')(outputs)
outputs = GlobalAveragePooling2D()(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(256)(outputs)
outputs = LeakyReLU(alpha=0.1)(outputs)
outputs = Dropout(0.25)(outputs)
outputs = Dense(num_classes, activation = 'softmax')(outputs)

model = Model(inputs = [inputs], outputs = [outputs])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [0]:
from keras.callbacks import *

weight_path=os.path.join( './Fine Tuning with pre-Trained VGG16 for Tuberculosis Classification', '{}_pretrainedvgg16model.bestweights.hdf5'.format('TB'))
checkpoint = ModelCheckpoint(weight_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="'val_loss'", mode="min", patience=30) 

callbacks_list = [checkpoint, early, reduceLROnPlat]

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size
model.fit_generator(train_gen, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_gen, validation_steps=STEP_SIZE_VALID,
                    epochs = 100, callbacks = callbacks_list)

Load Saved Model Weights and Perform Prediction with Heatmaps


In [0]:
import os
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda
from keras.layers import BatchNormalization, LeakyReLU
from keras.models import Model

baseDir =  '/content/drive/My Drive/Colab Notebooks/Tuberculosis-MIDL-NCAI/Tuberculosis Classification'
weight_path=os.path.join(baseDir, "{}_pretrainedvgg16model.bestweights.hdf5".format('Tuberculosis'))

IMG_SIZE = 512
img_shape = (IMG_SIZE, IMG_SIZE, 3)
inputs = Input(img_shape)
num_classes = 2

vgg16_model = VGG16(input_shape =  img_shape, include_top = False, weights = 'imagenet')
vgg16_model.trainable = False
outputs = vgg16_model(inputs)
outputs = BatchNormalization(name = 'BatchNormalization')(outputs)
outputs = GlobalAveragePooling2D()(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(256, activation = LeakyReLU(alpha=0.1))(outputs)
outputs = Dropout(0.25)(outputs)
outputs = Dense(num_classes, activation = 'sigmoid')(outputs)

model = Model(inputs = [inputs], outputs = [outputs])
model.load_weights(weight_path)

model.summary()

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 16, 16, 512)       14714688  
_________________________________________________________________
BatchNormalization (BatchNor (None, 16, 16, 512)       2048      
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0   

In [0]:
from keras import backend as K
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array



validDir = os.path.join(baseDir, 'ExtractedLungsData', 'Test', '1')
testImages = os.listdir(validDir)

for eachTestImg in (testImages):
  tbImg = img_to_array(load_img(os.path.join(validDir, eachTestImg)))
  tbImg = np.float32(tbImg/255.0)
  tbImg = tbImg.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
  preds = model.predict(tbImg)
  class_idx = np.argmax(preds[0])
  print('{}:{}'.format(eachTestImg, class_idx))
  class_output = model.output[:, class_idx]
  last_conv_layer = model.get_layer("BatchNormalization")

  grads = K.gradients(class_output, last_conv_layer.output)[0]
  pooled_grads = K.mean(grads, axis=(0, 1, 2))
  iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
  pooled_grads_value, conv_layer_output_value = iterate([tbImg])
  for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

  heatmap = np.mean(conv_layer_output_value, axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)

  origImg = img_to_array(load_img(os.path.join(validDir, eachTestImg)))
  heatmap = cv2.resize(heatmap, (origImg.shape[0], origImg.shape[1]))
  heatmap = np.uint8(255 * heatmap)
  origImg = np.uint8(255 * origImg)

  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
  heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
  superimposed_img = cv2.addWeighted(origImg, 1.0, heatmap, 0.7, 0)
  plt.figure(figsize = (8, 8))
  plt.subplot(1, 2, 1)
  plt.imshow(heatmap)
  plt.subplot(1, 2, 2)
  plt.imshow(superimposed_img, vmin=0, vmax=255)
  plt.show()
  
  

Output hidden; open in https://colab.research.google.com to view.